In [4]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import re
from sklearn.model_selection import train_test_split

stop_words = open('stopwords.txt', encoding='utf-8').read().split()

[nltk_data] Downloading package punkt to /home/yohannes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
cleaned_data = pd.read_csv('../cleaned_data.csv')

In [7]:
cleaned_data.head(10)

,title,content
0,የአማራ ብሄራዊ ክልል መንግስት መግለጫ,የሀገራችን ህዝቦች በአብሮነትና በአንድነት በህብረ ብሄራዊነት ለበርካታ አ...
1,አዴፓዎች ለዶክተር አብይ ግልፅ ቅድመ ሁኔታዎች ማስቀመጥ አለባቸው ግርማ ካሳ,በታገቱ ተማሪዎችና በሌሎች የአገር ደህንነትና መከላከያ ጉዳዮች ዙሪያ አዴ...
2,በተማሪዎች ላይ የተፈፀመውን የእገታ ድርጊት የሚያወግዙ ሰላማዊ ሰልፎች በ...,ባህር ዳር ሰላማዊ ሰልፎቹ በደምቢ ዶሎ ዩኒቨርሲቲ ሲማሩ የነበሩና የታገቱ...
3,በአቶ ገዱ አንዳርጋቸው የተመራ የልኡካን ቡድን ወደ አሜሪካ ተጓዘ,በኢፌዲሪ የውጭ ጉዳይ ሚኒስትር አቶ ገዱ አንዳርጋቸው የተመራ የኢትዮጵያ ...
4,ጦቢያ ግጥምን በጃዝ መላኩ ታረቀኝ,ጦቢያ ግጥምን በጃዝ መላኩ ታረቀኝ
5,ዶክተር አብይ ሱዳንን አሻገረ የሀገሬውም ህዝብ ለምስጋና እጅ ነሳው,ዶክተር አብይ ሱዳንን አሻገረ የሀገሬውም ህዝብ ለምስጋና እጅ ነሳው
6,በነገራችን ላይ አንዳርጋቸው ፅጌ,በነገራችን ላይ አንዳርጋቸው ፅጌ
7,ጠቅላይ ሚኒስትር ዶክተር አቢይ አህመድ ከህግ ባለሙያዎች ጋር ውይይት አካሄዱ,ጠቅላይ ሚኒስትር ዶክተር አቢይ አህመድ ከህግ ባለሙያዎች ጋር ውይይት አካሄዱ
8,የአሜሪካ ድምፅ ራዲዮ ከአቶ ንጉሱ ጥላሁን እና ከሆኑት አቶ ነቢያት ጌታቸ...,የአሜሪካ ድምፅ ራዲዮ የጠቅላይ ሚኒስትር ፅህፈት ቤት ፕሬስ ሴክሬታሪያት ...
9,ይድርስ በኢትዮጵያ ለምትገኙ የዩኒቨርስቲ ተማሪዎች,በገክርስቶስ አባይጥር ቀን አመተ ምህረትከሁሉ አስቀድሜ ሰላምና ጤና እ...


In [8]:
class CosineModel:

    am_sent_endings = r'\?|\!|\።|\፡፡'
    am_punctuation = '፠፡።፣፤፥፦፧፨“”‘’…‹‹››·•'
    am_numbers = '፩፪፫፬፭፮፯፰፱፲፳፴፵፶፷፸፹፺፻፼'
    am_random = '�©\uf0c4\uf0d8\uf0a7\uf066\uf0d8'
    stop_words = open('stopwords.txt', encoding='utf-8').read().split()

    def __init__(self, text):
        self.raw_text = text
        self.clean_text = None
        self.sentences = None
        self.words = None

        self.process_text()

    def process_text(self):
        self.clean_text = self.clean_minimized(self.raw_text)
        self.sentences = self.extract_sentences(self.clean_text)
        self.sentences = self.remove_duplicate_sentence(self.sentences)
        self.words = self.extract_words(self.clean_text)

    def extract_sentences(self, text=None):
        if text is None:
            text = self.raw_text
        sentences = re.split(self.am_sent_endings, text)
        return sentences

    def extract_words(self, text):
        return text.split()

    def clean_minimized(self, text):
        words = text.split()
        to_clean = string.punctuation + self.am_numbers + self.am_random + string.ascii_letters + string.digits + self.am_punctuation
        to_clean = re.sub(self.am_sent_endings, '', to_clean)
        table = str.maketrans('', '', to_clean)
        stripped = [w.translate(table) for w in words]
        clean_txt = list(filter(None, stripped))
        return ' '.join(clean_txt)

    def remove_duplicate_sentence(self, sentences):
        duplicates = []
        cleaned = []
        for s in sentences:
            if s in cleaned:
                if s in duplicates:
                    continue
                else:
                    duplicates.append(s)
            else:
                cleaned.append(s)
        return cleaned


In [9]:
! pip install etnltk

Defaulting to user installation because normal site-packages is not writeable


In [10]:
# def find_similarities(text):
#     sentences = remove_stopwords(text)
#     sentences = sent_tokenize(str(text))
#     print(sentences)
 
#     vectorizer = TfidfVectorizer()
   
#     trsfm = vectorizer.fit_transform(sentences)
#     text_df = pd.DataFrame(trsfm.toarray(), columns=vectorizer.get_feature_names_out(), index=sentences)

#     num_sentences = text_df.shape[0]
#     num_summary_sentences = int(np.ceil(num_sentences ** 0.5))

#     similarities = cosine_similarity(trsfm, trsfm)

#     avgs = []
#     for i in similarities:
#         avgs.append(i.mean())

    
#     top_idx = np.argsort(avgs)[-num_summary_sentences:]

#     return top_idx

In [11]:
# def build_summary(text):
#     sents_for_sum = find_similarities(text)
#     sort = sorted(sents_for_sum)
   
#     print('Number of selected sentences', len(sort))
#     sent_list = sent_tokenize(str(text))
#     print('Total number of sentences', len(sent_list))

#     sents = []
#     for i in sort:
#         sents.append(sent_list[i].replace('\n', '።') + '።')

#     summary = ' '.join(sents)
#     return summary
    

In [12]:
def find_similarities(sentences, stopwords):
    vectorizer = TfidfVectorizer(stop_words=stopwords)
    trsfm = vectorizer.fit_transform(sentences)
    text_df = pd.DataFrame(trsfm.toarray(), columns=vectorizer.get_feature_names_out(), index=sentences)
    num_sentences = text_df.shape[0]
    num_summary_sentences = int(np.ceil(num_sentences ** .5))
    similarities = cosine_similarity(trsfm, trsfm)
    
    avgs = []
    for i in similarities:
        avgs.append(i.mean())
 
    top_idx = np.argsort(avgs)[-num_summary_sentences:]
   
    return top_idx

In [25]:
def build_summary(sentences, stop_words):
    sents_for_sum = find_similarities(sentences, stopwords=stop_words)
    sort = sorted(sents_for_sum)
    print('Number of selected sentences', len(sort))
    sent_list = sentences
    print(sentences)
    print('Total number of sentences', len(sent_list))
   
    sents = []
    for i in sort:
        sents.append(sent_list[i].replace('\n', '') + '።')
    summary = ' '.join(sents)
    
    return summary

In [14]:
import etnltk
from etnltk.tokenize.am import word_tokenize

In [15]:
from difflib import SequenceMatcher

def longest_common_subsequence(a, b):
    matcher = SequenceMatcher(None, a, b)
    match = matcher.find_longest_match(0, len(a), 0, len(b))
    return a[match.a: match.a + match.size]


In [16]:
def rouge_n(reference, candidate, n=1):
    reference_tokens = word_tokenize(reference)
    candidate_tokens = word_tokenize(candidate)
    
    reference_ngrams = set(nltk.ngrams(reference_tokens, n))
    candidate_ngrams = set(nltk.ngrams(candidate_tokens, n))
    
    precision = len(reference_ngrams.intersection(candidate_ngrams))
    recall = len(reference_ngrams) + len(candidate_ngrams) - precision
    
    if recall == 0:
        rouge_n_score = 0.0
    else:
        rouge_n_score = precision / recall
    
    return rouge_n_score

In [17]:
def rouge_l(reference, candidate):
    reference_tokens = word_tokenize(reference)
    candidate_tokens = word_tokenize(candidate)
    
    reference_ngrams_all = list(nltk.ngrams(reference_tokens, 1))
    candidate_ngrams_all = list(nltk.ngrams(candidate_tokens, 1))
    
    reference_lcs = longest_common_subsequence(reference_ngrams_all, candidate_ngrams_all)
    
    rouge_l_score = len(reference_lcs) / max(1, len(reference_ngrams_all))
    
    return rouge_l_score


In [18]:
def calculate_rouge_scores(reference, candidate):
    rouge_n_1 = rouge_n(reference, candidate, n=1)
    rouge_n_2 = rouge_n(reference, candidate, n=2)
    rouge_l_score = rouge_l(reference, candidate)
    
    return rouge_n_1, rouge_n_2, rouge_l_score

In [19]:
def rouge_score(reference_summary, generated_summary):
    rouge_n_1, rouge_n_2, rouge_l_score = calculate_rouge_scores(reference_summary, generated_summary)
    
    print(f"ROUGE-1: {rouge_n_1:.4f}")
    print(f"ROUGE-2: {rouge_n_2:.4f}")
    print(f"ROUGE-L: {rouge_l_score:.4f}")


In [20]:
X = cleaned_data['content']
y = cleaned_data['title']    

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
content = '''
በአፍሪካ ዋንጫ ናይጄሪያ በአዴሞላ ሉክማን ሁለት ጎሎች ካሜሩንን በመርታት ወደ ሩብ ፍፃሜው ተሸጋገረች። ደካማ የሚባል አቋም ያሳየችው ካሜሩን ሻንጣዋን ልትሸክፍ ግድ ሆኗል። የጣሊያኑ አታላንታ አጥቂ ለአጋማሽ ዘጠኝ ደቂቃዎች ሲቀሩት ባስቆጠራት ጎል ሀገሩን ቀዳሚ አድርጓል። 
ሉክማን ከናፖሊው አጥቂ ቪክተር ኦሲምሄን ያገኘውን ኳስ ኦናናን ተክቶ የተሰለፈው ግብ ጠባቂው ፋብሪስ ኦንደዋን ተሻግሮ አስቆጥሯል። የቀድሞው ከ21 ዓመት በታች የእንግሊዝ ብሔራዊ ቡድን አጥቂ የነበረው ሉክማን ሁለተኛዋን ጎሎ በ90ኛው ደቂቃ በማስቆጠር የካሜሩንን ልብ ሰብሯል።
ምንም እንኳ በጉዳት ምክንያት ሳይሰለፍ የቆየው የካሜሩኑ አጥቂ ቪንሴንት አቡበከር ዘግይቶ ወደ ሜዳ ቢገባም ሀገሩ አንድ ዒላማውን የጠበቀ ኳስ ሳታስመዘግብ ወጥታለች። አቡበከር ባለፈው ካሜሩን ባዘጋጀችው የ2021 የአፍሪካ ዋንጫ ኮከብ ጎል አስቆጣሪ ነበር። በዓለም ዋንጫ ከምድቡ ማለፍ ያልቻለው የካሜሩኑ አሠልጣኝ ሪጎቤርት ሶንግ ጫናው እየበረታ መጥቷል።
የካሜሩን እግር ኳስ ፌዴሬሽን ፕሬዝደንት ሳሙዔል ኤቶ ጨዋታውን በሥፍራው ተገኝቶ የተከታተለ ሲሆን የቀድሞ የቡድን አጋሩን ሥራ የመታደግ አሊያም የማባረር ኃላፊነት አለበት። በዘንድሮው ውድድር እምብዛም አስደናቂ ብቃት ያላሳዩት የማይበገሩት አናብስት ከናይጄሪያ በነበራቸውም ጨዋታው ከቅርፅ ውጭ ሆነው ተስተውለዋል።
በተቃራኒው ናይጄሪያ ያገኘችው ውጤት የሚገባት የሚያስብል ሲሆን ተጋጣሚዋን በልጣ ታይታለች። በዘጠነኛው ደቂቃ የናይጄሪያው ተከላካይ ሴሚ አጃይ ያስቆጠራት ጎል በቪኤአር ታይታ ውድቅ ሆናበታለች።
የካሜሩንን የመጀመሪያ ጨዋታ ለማንቸስተር ዩናይትድ ለመጫወት ሲል የቀጣው ግብ ጠባቂው አንድሬ ኦናና ለቀሪዎቹ ጨዋታዎች በግል አውሮፕላን ወደ አይቮሪ ኮስት ቢበርም መሰለፍ የቻለው አንድ ጨዋታ ብቻ ነው። እሱን ተክቶ የተሰለፈው ኦንዶዋ ስኅተት በፈፀመ ቁጥር ካሜራው ኦናናን ሲያስመልክት ተስተውሏል።
በሌላ ጨዋታ አንጎላ ሁለት ተጫዋች በቀይ ያጣችውን ናሚቢያን 3 ለምንም በመርታት ወደ ሩብ ፍፃሜው ተቀላቅላለች። የናሚቢያው ግብ ጠባቂ ኔብሉ በ17ኛው ደቂቃ በቀጥታ ቀይ ካርድ ከሜዳ ተሰናብቷል። ጌልሰን ዳላ ከ9 ደቂቃ በኋላ ለአንጎላ ጎል ካስቆጠረ በኋላ የናሚቢያው ተከላካይ ሉቤኒ ሃውኮንጎ በሁለት ቢጫ ካርድ ከሜዳ ተባሯል።
ዳላ ለራሱ እና ለሀገሩ ሁለተኛዋን ጎል ሲያስቆጥር ማቡሉሉ ደግሞ በአስደናቂ ሁኔታ ሶስተኛዋን በማከል አንጎላ ሩብ ፍፃሜውን እንድትቀላቀል አስችለዋል። አንጎላ በመጪው አርብ በሩብ ፍፃሜው ናይጄሪያን ትገጥማለች።
        '''

title  = '''
 በአፍሪካ ዋንጫ ናይጄሪያ በአዴሞላ ሉክማን ሁለት ጎሎች ካሜሩንን በመርታት ወደ ሩብ ፍፃሜው ተሸጋገረች። 
 ሉክማን ከናፖሊው አጥቂ ቪክተር ኦሲምሄን ያገኘውን ኳስ ኦናናን ተክቶ የተሰለፈው ግብ ጠባቂው ፋብሪስ ኦንደዋን ተሻግሮ አስቆጥሯል።  
 በሌላ ጨዋታ አንጎላ ሁለት ተጫዋች በቀይ ያጣችውን ናሚቢያን ለምንም በመርታት ወደ ሩብ ፍፃሜው ተቀላቅላለች።  ጌልሰን ዳላ ከ ደቂቃ በኋላ ለአንጎላ ጎል ካስቆጠረ በኋላ የናሚቢያው ተከላካይ ሉቤኒ ሃውኮንጎ በሁለት ቢጫ ካርድ ከሜዳ ተባሯል።
       '''
tparser = CosineModel(content)
article_summary = build_summary(tparser.sentences, tparser.stop_words)
rouge_score(title, article_summary)
print("\n\n")
print("labeled summary", title)
print("\n")
print("Generated Summary", article_summary)


Number of selected sentences 5
['በአፍሪካ ዋንጫ ናይጄሪያ በአዴሞላ ሉክማን ሁለት ጎሎች ካሜሩንን በመርታት ወደ ሩብ ፍፃሜው ተሸጋገረች', ' ደካማ የሚባል አቋም ያሳየችው ካሜሩን ሻንጣዋን ልትሸክፍ ግድ ሆኗል', ' የጣሊያኑ አታላንታ አጥቂ ለአጋማሽ ዘጠኝ ደቂቃዎች ሲቀሩት ባስቆጠራት ጎል ሀገሩን ቀዳሚ አድርጓል', ' ሉክማን ከናፖሊው አጥቂ ቪክተር ኦሲምሄን ያገኘውን ኳስ ኦናናን ተክቶ የተሰለፈው ግብ ጠባቂው ፋብሪስ ኦንደዋን ተሻግሮ አስቆጥሯል', ' የቀድሞው ከ ዓመት በታች የእንግሊዝ ብሔራዊ ቡድን አጥቂ የነበረው ሉክማን ሁለተኛዋን ጎሎ በኛው ደቂቃ በማስቆጠር የካሜሩንን ልብ ሰብሯል', ' ምንም እንኳ በጉዳት ምክንያት ሳይሰለፍ የቆየው የካሜሩኑ አጥቂ ቪንሴንት አቡበከር ዘግይቶ ወደ ሜዳ ቢገባም ሀገሩ አንድ ዒላማውን የጠበቀ ኳስ ሳታስመዘግብ ወጥታለች', ' አቡበከር ባለፈው ካሜሩን ባዘጋጀችው የ የአፍሪካ ዋንጫ ኮከብ ጎል አስቆጣሪ ነበር', ' በዓለም ዋንጫ ከምድቡ ማለፍ ያልቻለው የካሜሩኑ አሠልጣኝ ሪጎቤርት ሶንግ ጫናው እየበረታ መጥቷል', ' የካሜሩን እግር ኳስ ፌዴሬሽን ፕሬዝደንት ሳሙዔል ኤቶ ጨዋታውን በሥፍራው ተገኝቶ የተከታተለ ሲሆን የቀድሞ የቡድን አጋሩን ሥራ የመታደግ አሊያም የማባረር ኃላፊነት አለበት', ' በዘንድሮው ውድድር እምብዛም አስደናቂ ብቃት ያላሳዩት የማይበገሩት አናብስት ከናይጄሪያ በነበራቸውም ጨዋታው ከቅርፅ ውጭ ሆነው ተስተውለዋል', ' በተቃራኒው ናይጄሪያ ያገኘችው ውጤት የሚገባት የሚያስብል ሲሆን ተጋጣሚዋን በልጣ ታይታለች', ' በዘጠነኛው ደቂቃ የናይጄሪያው ተከላካይ ሴሚ አጃይ ያስቆጠራት ጎል በቪኤአር ታይታ ውድቅ ሆናበታለች', ' የካሜሩንን የመጀመሪያ ጨዋታ ለማንቸስተር ዩናይትድ ለመጫወት ሲል የቀጣው ግ

In [ ]:
input_text = '''
የተቀናጀ (Integrated) የሚለው ቃል በጥሬው ሲተረጎም፣ የተለያዩ ነገሮችን በማጣመር ወይም በማዋሃድ የተስማማና ግንኙነቱ የጠበቀ ሙሉ ነገርን መፍጠር ማለት ነው፡፡ ወደ ተቀናጀ ቅድመ አንደኛ ደረጃ ትምህርት ስንመጣ የቅንጅቱ መነሻ፣ የተቀረፁት ጭብጦችና ለዚህ ማስተላለፊያ ተብለው የተዘጋጁት የመማር ዓውዶች ይሆናል፡፡ በዚህ ሒደት ስለቅንጅት ስናወራ በቀጥታ ወይም በተዘዋዋሪ መንገድ ስለጭብጥና የመማር ዓውድ እናወራለን ማለት ነው፡፡ ስለዚህ የቅድመ አንደኛ ደረጃ ሥርዓተ ትምህርት ቅንጅት በአብዛኛው የሚወድቀው በጭብጡና በመማር ዓውዱ፣ እንዲሁም በመማር ዓውዱ ንዑሳን ክፍሎች መካከል ይሆናል ማለት ነው፡፡ ከዚህ የሚነሱ የተለያዩ የቅንጅት ዓይነቶች ይገኛሉ፡፡ ከነዚህም ውስጥ፣ በቅድመ አንደኛ ደረጃ ትምህርት ሥርዓተ ትምህርት አደረጃጀት ውስጥ በድግግሞሽ የሚነሱትና የሚተገበሩት ሁለት ናቸው፡፡ እነሱም፣ ውጫዊ ቅንጅት (Inter-disciplinary) እና ውስጣዊ ቅንጅት (Intra-Disciplinary) የሚባሉት ናቸው፡፡ 

'''
# input_text = "ተዘግቶ የነበረው መጋረጃ ሲገለጥ የሚል ርእስ የሰጠሁበት ዋናው ምክንያት እነ ሻምበል አምሃ አበበና የመሳሰሉት በመቶዎች የሚቆጠሩ ብዙ ወጣት መኮንኖች የበታች ሹሞችና ወታደሮች በኮርስ ወይም በቅርብ ጓደኞቻቸው የሀሰት ጥቆማ ተካሂዶባቸው ህይወታቸው የተቀጨበት አካለ ስንኩላን የሆኑበት ምክንያት ብዙ ሚስጢር መውጣት በመጀመሩ ነው። ኮሎኔል መርሻ ወዳጆ በመጽሐፉ ሽፋን ላይ ስለመጽሐፉ የሚከተለውን አስተያየት ሰጥተዋል። ጄኔራል አማን የጦር አለቆችን ማነጋገር መጀመራቸው የደርግ አባላቱ ሲረዱ የፍርሃትና የጭንቀት ብርድ ውስጥ ከተታቸው። ደህንነቱ ልዩ ልዩ የጦር ክፍሎችና ከፖሊስ ሠራዊት ተዛውረን የመጣን ሲሆን ከመሥሪያ ቤቴ የሰው ኃይል ሲነፃፀር ብዛት ባይኖራቸውም በተለያዩ የትምህርት ዘርፎች በዶክትሬት በማስትሬት በባችለር ዲግሪ ከዚያም ዝቅ ሲል በዲፕሎማ የተመረቁ በተጨማሪ ብዛት ያላቸው ሠራተኞች ወደ ተለያዩ ሶሻሊስት አገሮች ተልከው የመረጃና የሶሻሊስት ርእዮተ ዓለም ትምህርት የተክታተሉ አባሎች እንደነበሩ እሙን ነው። በሰሜን በኩል የነበረው የኢትዮጵያ ጦር መፈረካከስና መበተን የጀመረው ከግንቦት ቀን በኋላ እንደሆነ ግልጽ ነው። አስተያየት በእርግጥም የውጭ መረጃ ሠራተኞች ጂቡቲ ሆነው የሱማሌን መንግሥት የጦር ቢሮ ቦርቡረው ውጤት አስገኝተዋል የሚለው እውነትነት ካለው ሁላችንንም ማለትም በደህነነት መቤት ውስጥ የነበርነውን ሠራተኞች ሁሉ የሚያኮራን ነው። ከ ዓም ወዲህ የሻዕቢያና የ ወያኔ ቡድን አባሎች በጋራና በተናጠል አገር ውስጥ በስውር ይንቀሳቀሱ እንደነበር በተመሳሳይ ጂቡቲ ኬኒያ ሱማሌና በተለይም ሱዳን ውስጥ ጽሕፈት ቤቶቻቸውን ከፍተው በከፍተኛ ደረጃ ፀረ ኢትዮጵያ እንቅስቃሴ ሲያደርጉ እንደነበር ይታወቃል። ሻምበል ተስፋዬ ርስቴ በጻፉት መጽሐፍ በገጽ የውጭ አገሮች ጥናትና ምርምር ድርጅት ውስጥ እየተገናኙ ውይይት ያደርጉ እንደነበር ገልጸዋል። ስለዚህ የፕሬዚዳንት መንግሥቱ ኃማርያም ማንነትና የሚስጥር አማካሪዎቻቸው እነማን እንደነበሩ ከላይ የተገለጸው ጥሩ ማስረጃ ነው እላለሁ። በእርግጥ ከላይ ስማችሁ የተገለጸው መኮንኖች የአንድ ኮርስ መኮንኖች መሆናችሁን በወቅቱ ሻምበል ቁምላቸው ተካና ሻምበል አመሀ አበበ አንድ ቤት ውስጥ ተከራይተው ይኖሩ እንደነበር ይነገራል።"
# input_text = "በኢትዮጵያ በጥሬ ገንዘብ የሚደረጉ የገንዘብ እንቅስቃሴዎች እየቀነሱ መሆኑን መረጃዎች እያመለከቱ ነው፡፡ በአብዛኛዎቹ የኢትዮጵያ ባንኮች የሚደረጉ የገንዘብ እንቅስቃሴዎች በተለያዩ የዲጂታል አማራጮች እየተተገበሩ ነው፡፡ በተለይ በ2015 ሒሳብ ዓመት እንደ ሪከርድ የሚቆጠር አፈጻጸም መታየቱም ይገልጻል፡፡ በ2016 ሒሳብ ዓመትም ቢሆን በዲጂታል ባንክ አገልግሎት የሚንቀሳቀሱ የገንዘብ መጠን እያደገ ስለመምጣቱ እየወጡ ያሉ መረጃዎች እያመላከቱ ነው፡፡ በጥሬ ገንዘብ የሚደረግ ግብይት በከፍተኛ ደረጃ እየቀነሰ መምጣቱ ተጠቆመ:: የኢትዮጵያ ብሔራዊ ባንክ ይፋዊ መረጃ እንደሚያሳየውም፣ በተጠናቀቀው የ2015 ሒሳብ ዓመት በዲጂታል የክፍያ አማራጮች የተንቀሳቀሰው የገንዘብ መጠን ከ4.7 ትሪሊዮን ብር በላይ ደርሷል፡፡ ይህ የገንዘብ መጠን ከቀዳሚው ዓመት የ2014 የሒሳብ ዓመት ጋር ሲነፃፀር፣ ከ3.1 ትሪሊዮን ብር በላይ ብልጫ አለው፡፡ በግብይት መጠንም ቢሆን በ2014 የሒሳብ ዓመት የተፈጸመው ትራንዛክሽን ወይም ግብይት 345.7 ሚሊዮን ሲሆን፣ በ2015 ሒሳብ ዓመት ግን 1.24 ቢሊዮን ደርሷል፡፡ "
# input_text = "በኢትዮጵያ ንግድ ባንክ መረጃ መሠረት በ2016 የሒሳብ ዓመት ግማሽ ዓመት ብቻ ከ2.8 ትሪሊዮን ብር በላይ በዲጂታል የባንክ አገልግሎት ገንዘብ ማንቀሳቀስ ችሏል፡፡ ይህ ገንዘብ እንቅስቃሴ ከ494 ሚሊዮን ብር በላይ በሚሆኑ ግብይቶች ወይም ትራንዛሽኖች የተፈጸመ ነው፡፡ ይህ አፈጻጸም ከቀዳሚው ዓመት ተመሳሳይ ወቅት ጋር ሲነፃፀር፣ በግብይት የ35 በመቶ በገንዘብ መጠን ደግሞ የ118 በመቶ ዕድገት የታየበት ነው፡፡ በቀዳሚው ዓመት በሙሉ የሒሳብ ዓመት 3.2 ትሪሊዮን ብር ማንቀሳቀስ ችሎ የነበረው የኢትዮጵያ ንግድ ባንክ፣ የስድስት ወራት 2.8 ቢሊዮን ብር ማንቀሳቀስ የቻለው ከ12 ሚሊዮን በላይ የኤቲኤም ተጠቃሚ፣ ከዘጠኝ ሚሊዮን በላይ የሞባይል ባንክና ከ20 ሚሊዮን ብር በላይ በሚሆኑ ደንበኞች በመያዝ እንደሆነ ታውቋል፡፡"

tparser = CosineModel(input_text)
article_summary = build_summary(tparser.sentences, tparser.stop_words)

print("\n\n")
print("Text")
print("\n")
print(input_text)
print("===================")
print("Summarized")
print("\n")
print(article_summary)